In [1]:
from cmath import nan
from math import isnan
import difflib
from tkinter.tix import COLUMN 
import pandas as pd
import numpy as np
import json
from pathlib import Path
import lib.pdfield as pdfield


In [2]:
### Setup connection to Field-database-server based on config.json file ###
config = pdfield.loadconfigs('config\config.json')
api = pdfield.couchDB_APIs(config)

In [3]:
### Import all documents from Target Field-database ###
targetDOCs= pdfield.getAllDocs(api)
targetDF, docfields  = pdfield.allDocsToDf(targetDOCs)


The database uruk contains 19904 docs.
This is round 1offset : 0
This is round 2offset : 10000
19904
['created', 'modified', '_id', '_rev', 'project', 'resource.identifier']


In [4]:
%reload_ext autoreload
%autoreload 2

### Import csv-file with data to be integrated ###
importcsv_path = 'C:/Users/mhaibt/Nextcloud2/Uruk/WarkaEnvironsSurvey/WES_collections/WES_potterycollectionsdata_beforeidaifield_complete_MASTERVERSION.csv'
importcsvDF = pd.read_csv(importcsv_path, sep=',' ) 
### Rename columns of csv-file-table according to mapping-json  to match does used in Field ###
importcsvDF = importcsvDF.rename(columns=pdfield.loadconfigs('example_mappings_messydata/columnMapping_Uruk_dbf2field.json'))


In [5]:
%reload_ext autoreload
%autoreload 2

importcsvDF2 = importcsvDF

### Create a Uuid for rows from csv-file-table and set it as index ###
importcsvDF2 = importcsvDF2.apply(pdfield.makeUuid, fieldname='myid', axis=1)
importcsvDF2.set_index('myid',drop=False,inplace=True)

### Detrmine primary 'type'(Field-semantic) of input csv-file-table
importcsvDF2['type']='Pottery'

### Simple datatype manipulations on data from csv-file-table ###
amount = importcsvDF2[importcsvDF2['amount'].notnull()].apply(pdfield.commaNumber2int, inputfield = 'amount', axis=1)
amount.set_index('myid',drop=False,inplace=True)
importcsvDF2.update(amount)
importcsvDF2['amount'] = importcsvDF2['amount'].fillna(1).astype(int)
conditionPercent = importcsvDF2[importcsvDF2['conditionPercent'].notnull()].apply(pdfield.commaNumber2int, inputfield = 'conditionPercent', axis=1)
conditionPercent.set_index('myid',drop=False,inplace=True)
importcsvDF2.update(conditionPercent)
importcsvDF2['REC_NO_']=importcsvDF2['REC_NO_'].fillna(0).astype(int)
manufacturingMethod = importcsvDF2[importcsvDF2['manufacturingMethod'].notnull()].apply(pdfield.singleStrfield2List, inputfield='manufacturingMethod', axis=1)
manufacturingMethod.set_index('myid',drop=False,inplace=True)
importcsvDF2.update(manufacturingMethod)
Firing = importcsvDF2[importcsvDF2['Firing'].notnull()].apply(pdfield.singleStrfield2List, inputfield='Firing', axis=1)
Firing.set_index('myid',drop=False,inplace=True)
importcsvDF2.update(Firing)
coatOutsideType = importcsvDF2[importcsvDF2['coatOutsideType'].notnull()].apply(pdfield.singleStrfield2List, inputfield='Firing', axis=1)
coatOutsideType.set_index('myid',drop=False,inplace=True)
coatOutsideType.update(Firing)

### Convert/Format input Date to FindDate ###
importcsvDF2['date']=np.nan
dateDF = importcsvDF2.apply(pdfield.format2FindDate,inputfield='identifier', outputfield='date', axis=1)
dateDF.set_index('myid',drop=False,inplace=True)
importcsvDF2.update(dateDF)


### Change input values according to strict value Mappings determined in external json ###
mappings=pdfield.loadconfigs('example_mappings_messydata/fieldvalueMappings_Uruk_dbf2field.json')
importcsvDF2['processor'] = importcsvDF2['processor'].map(mappings['personalMap']).fillna(importcsvDF2['processor'])
importcsvDF2['temper'] = importcsvDF2['temper'].map(mappings['temperMap']).fillna(importcsvDF2['temper'])
importcsvDF2['temperAmount'] = importcsvDF2['temperAmount'].map(mappings['temperAmountMap']).fillna(importcsvDF2['temperAmount'])
importcsvDF2['temperParticles'] = importcsvDF2['temperParticles'].map(mappings['temperParticlesMap']).fillna(importcsvDF2['temperParticles'])

### Change complex input values of type string to normed item-lists Part of vessel ###
vesselpart = importcsvDF2[importcsvDF2['vesselpart'].notnull()].apply(pdfield.strings2splitlistitems,inputfield='vesselpart',replacemap={' ':'','?':''},regex=';|,|-|or|oder|with|\*|\n', axis=1)
vesselpart.set_index('myid',drop=False,inplace=True)
importcsvDF2.update(vesselpart)
importcsvDF2 = importcsvDF2.apply(pdfield.adaptListitemsByMap,inputfield='vesselpart',normmap=mappings['vesselpartMap'], axis=1)



### Change complex input values of type string to normed item-lists Period ###
importcsvDF2 = importcsvDF2.apply(pdfield.strings2splitlistitems,inputfield='period',replacemap={' ':'','I/I':'I-I','L/o':'L-o','ED.Akkad':'ED-Akkad'},regex=';|,|-|or|oder|bis|[^"I/L"]/|\*|\n', axis=1)
importcsvDF2 = importcsvDF2.apply(pdfield.adaptListitemsByMap,inputfield='period',normmap=mappings['periodMap'], axis=1)
importcsvDF2 = importcsvDF2.apply(pdfield.formatListitems2periodvalues,inputfield='period',axis=1)



### Convert simple Diameter value to dimensionDiameter ###
importcsvDF2 = importcsvDF2.apply(pdfield.simpleDiameter2dimensionDiameter, axis=1)
importcsvDF2.drop(columns=['DIAMETER_I'])








,DrawingId,FEATURE_NU,Firing,FormComparisonCompendiadetail,REC_NO_,SAMPLE_NUM,Unnamed: 21,amount,clayColorOutside,coatOutsideType,...,identifier,manufacturingMethod,myid,period,processor,temper,temperAmount,temperParticles,type,vesselpart
myid,,,,,,,,,,,,,,,,,,,,,
609ec13e-c783-450e-8754-2177534e322e,NaN,0253,[normal],BaM19_410_125,1.0,L18,NaN,1.0,red,wet sm,...,2018-02-07 13:32:00,[wheel],609ec13e-c783-450e-8754-2177534e322e,"{'value': 'Isin-Larsa', 'endValue': 'Old Babyl...","[Jan Hubert, Margarete van Ess]",[non clay minerals],occasional,medium-fine,Pottery,[rim sherd]
c4d2294b-0576-4602-b0aa-3f6997faa1d9,NaN,0253,[normal],OIP 129 Pl. 87_10,2.0,L18,NaN,1.0,yellow-green,wet sm,...,2018-02-07 13:32:00,[wheel],c4d2294b-0576-4602-b0aa-3f6997faa1d9,"{'value': 'Early Dynastic III', 'endValue': 'A...","[Jan Hubert, Margarete van Ess]",[non clay minerals],medium,medium-fine,Pottery,[rim sherd]
38f73f8d-d2c8-4f87-a0ce-573b5e87cf7d,NaN,0253,[normal],Typ. Salje 272_103 oder BaM 19_419_158,3.0,L18,NaN,1.0,red,wet sm,...,2018-02-07 13:32:00,[wheel],38f73f8d-d2c8-4f87-a0ce-573b5e87cf7d,[],"[Jan Hubert, Margarete van Ess]",[non clay minerals],medium,medium-fine,Pottery,[rim sherd]
dd7f752d-f595-41e0-a83f-9f7016cc6ba9,NaN,0253,[normal],"BaM19_304_W21420,3",4.0,L18,NaN,1.0,red,wet sm,...,2018-02-07 13:32:00,[wheel],dd7f752d-f595-41e0-a83f-9f7016cc6ba9,[],[Jan Hubert],[non clay minerals],medium,medium-fine,Pottery,[rim sherd]
3c7d2891-f160-438a-9dd5-23709d3c6946,Z1,0253,[normal],OIP 129 Pl. 122_3,5.0,NaN,NaN,1.0,brown,wet sm,...,2018-02-07 13:32:00,[wheel],3c7d2891-f160-438a-9dd5-23709d3c6946,"{'value': 'Early Dynastic III', 'endValue': 'O...","[Jan Hubert, Margarete van Ess]","[non clay minerals, organic particles]",frequent,medium,Pottery,[rim sherd]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678bdbde-9424-4842-8608-713fae4f75a5,Z2,NaN,[over fired],L10_north_G_westofslag_Z2,0.0,NaN,NaN,1.0,NaN,wet sm,...,L10_north_G_westofslag,NaN,678bdbde-9424-4842-8608-713fae4f75a5,"{'value': 'Middle Babylonian', 'endValue': 'Ne...",[Margarete van Ess],[organic particles],no inclusions,medium-fine,Pottery,[rim sherd]
d9c87cfd-1541-400c-a0fb-2deb22742c82,Z3,NaN,[over fired],L10_north_G_westofslag_Z3,0.0,NaN,NaN,1.0,green,wet sm,...,L10_north_G_westofslag,NaN,d9c87cfd-1541-400c-a0fb-2deb22742c82,"{'value': '3.Jt.', 'endValue': '3.Jt.'}",[Margarete van Ess],NaN,no inclusions,NaN,Pottery,[rim sherd]
af7ce280-52fe-4295-9ef8-d08323c14490,Z4,NaN,NaN,L10_north_G_westofslag_Z4,0.0,NaN,NaN,1.0,NaN,NaN,...,L10_north_G_westofslag,[mould],af7ce280-52fe-4295-9ef8-d08323c14490,"{'value': 'Uruk', 'endValue': 'Uruk'}",[Margarete van Ess],NaN,no inclusions,NaN,Pottery,[rim sherd]


In [12]:
%reload_ext autoreload
%autoreload 2
importcsvDF3 = importcsvDF2

### Drawings ###

importcsvDF3.set_index('myid',drop=False,inplace=True)
importfinds = importcsvDF3[importcsvDF3['DrawingId'].notnull()]
importfinds.set_index('myid',drop=False,inplace=True)
print('So many imported finds have a DrawingId: ', len(importfinds))
importfinds = importfinds.apply(pdfield.formatDrawing2DrawingId, axis=1)
drawings = targetDF[targetDF['type']=='Drawing']
print('So many Drawings in DB: ', len(drawings))

drawings.drop(columns='DrawingId', inplace=True)
drawings.rename(columns={'identifier':'DrawingId'}, inplace=True)

importfindscomb = pdfield.combineFramesByFixId(inputdf=drawings,targetdf = importfinds ,id ='DrawingId',combinefields = ['id'], replacemap={'.JPG':'','.png':'','.jpg':''})
importfindscomb.set_index('myid',drop=False,inplace=True)
importfindsfix = importfindscomb[importfindscomb['id'].notnull()]
importfindsfix.set_index('myid',drop=False,inplace=True)
print('imported finds found fixid Drawing: ', len(importfindsfix))

importfindsfix =  importfindsfix.apply(pdfield.createDictColumn,outputfield='relations',axis=1)
importfindsfix = importfindsfix.apply(pdfield.createRelationFromField, inputfield= 'id', relation ='isDepictedIn', axis=1)

importcsvDF3['relations']= np.nan
importcsvDF3.update(importfindsfix, overwrite=False)

# exclude from possible drawings those which got a match by fix
notfixedDrawings = drawings[~drawings['id'].isin(importfindsfix['id']) ]
importfindsfix.drop(columns=['id'], inplace=True)
print('So many Drawings left in DB: ', len(notfixedDrawings))
importfindsfixnot = importfindscomb[importfindscomb['id'].isnull()]
importfindsfixnot.set_index('myid',drop=False,inplace=True)
importfindsfixnot = pdfield.combineFramesByFuzzyId(targetdf=importfindsfixnot,inputdfex=notfixedDrawings,id='DrawingId',combinefields=['id'], replacemap={'.JPG':'','.png':'','.jpg':''}, fuzziness=0.75, oneonone=True)
importfindsfixnot.set_index('myid',drop=False,inplace=True)
importfindsfuzzy = importfindsfixnot[importfindsfixnot['id'].notnull()]
importfindsfuzzy.set_index('myid',drop=False,inplace=True)
print('imported finds found fuzzyid Drawing: ', len(importfindsfuzzy))

importfindsfuzzy =  importfindsfuzzy.apply(pdfield.createDictColumn,outputfield='relations',axis=1)
importfindsfuzzy = importfindsfuzzy.apply(pdfield.createRelationFromField, inputfield= 'id', relation ='isDepictedIn', axis=1)
importfindsfuzzy.drop(columns=['id'], inplace=True)
importcsvDF3.update(importfindsfuzzy, overwrite=False)

print('Rows after assign Drawings ', len(importcsvDF3))
relationlist2 = []
importcsvDF3[importcsvDF3['relations'].notnull()]['relations'].apply(lambda x: relationlist2.append(x.get('isDepictedIn')) )
#print(len(pdfield.flatten([i for i in relationlist2 if i is not None])))
print(len(list(set(pdfield.flatten([i for i in relationlist2 if i is not None])))))
print(importcsvDF3['relations'])


So many imported finds have a DrawingId:  1016
So many Drawings in DB:  3342
imported finds found fixid Drawing:  225
So many Drawings left in DB:  3120


c:\Users\mhaibt\.conda\envs\accessidaifield\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\mhaibt\.conda\envs\accessidaifield\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
c:\Users\mhaibt\Documents\GitHub\pandasField\lib\pdfield.py:396: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

imported finds found fuzzyid Drawing:  0
Rows after assign Drawings  6710
222
myid
609ec13e-c783-450e-8754-2177534e322e                                                  NaN
c4d2294b-0576-4602-b0aa-3f6997faa1d9                                                  NaN
38f73f8d-d2c8-4f87-a0ce-573b5e87cf7d                                                  NaN
dd7f752d-f595-41e0-a83f-9f7016cc6ba9                                                  NaN
3c7d2891-f160-438a-9dd5-23709d3c6946                                                  NaN
                                                              ...                        
678bdbde-9424-4842-8608-713fae4f75a5    {'isDepictedIn': ['974a7613-78e0-40a5-966b-dbb...
d9c87cfd-1541-400c-a0fb-2deb22742c82    {'isDepictedIn': ['c22eac79-eb95-4433-a885-3a6...
af7ce280-52fe-4295-9ef8-d08323c14490    {'isDepictedIn': ['92be2366-02d4-4b7f-b318-f97...
a6720d62-158c-4a10-91aa-efe707ae4e18    {'isDepictedIn': ['a8507faa-7772-40a2-af57-980...
2ded9d55-e344-40a

In [13]:
%reload_ext autoreload
%autoreload 2
importcsvDF4 = importcsvDF3
### Change colour values by fuzzymatching to the norm values ###
uniquecolors = targetDF['clayColorOutside'].apply(pd.Series).stack().reset_index(drop=True)
clayColorOutside = importcsvDF4[importcsvDF4['clayColorOutside'].notnull()].apply(pdfield.strings2splitlistitems,inputfield='clayColorOutside', axis=1)
clayColorOutside.set_index('myid',drop=False,inplace=True)
clayColorOutside= clayColorOutside.apply(pdfield.fuzzyAdaptListitemsByNormitems,inputfield='clayColorOutside',normitems=list(uniquecolors.unique()), fuzziness=0.85, axis=1)
importcsvDF4.update(clayColorOutside)

### Equalize ColorINside and Outside as it has not been differentiated in the input data ###
importcsvDF4['clayColorInside']=importcsvDF4['clayColorOutside']

### Change Coat pottery values by fuzzymatching to the norm values ###
uniquecoat = targetDF['coatOutsideType'].apply(pd.Series).stack().reset_index(drop=True)
importcsvDF4['DecorationComparisonCompendiadetail'] = np.nan
coatOutsideType = importcsvDF4[importcsvDF4['coatOutsideType'].notnull()]
move = coatOutsideType.apply(pdfield.moveValues2otherField, inputfield='coatOutsideType', outputfield='DecorationComparisonCompendiadetail', valuelist=mappings['coatOutsideTypeExclude'], axis=1)
coatOutsideType.update(move)
coatOutsideType = coatOutsideType.apply(pdfield.adaptListitemsByMap,inputfield='coatOutsideType',normmap=mappings['coatOutsideTypeMap'], axis=1)
coatOutsideType = coatOutsideType.apply(pdfield.fuzzyAdaptListitemsByNormitems,inputfield='coatOutsideType',normitems=list(uniquecoat.unique()), fuzziness=0.75, axis=1)
coatOutsideType.set_index('myid',drop=False,inplace=True)
importcsvDF4.update(coatOutsideType)


### Assign SurveyUnits ###
surveyunitsDF = targetDF[targetDF['type']=='SurveyUnit']
targetdf = importcsvDF4[importcsvDF4['identifier'].notnull()]
importcsvDF4fix = pdfield.combineFramesByFixId(surveyunitsDF,targetdf = targetdf ,id ='identifier',combinefields = ['id'])
importcsvDF4fixnot = importcsvDF4fix[importcsvDF4fix['id'].isnull()]
importcsvDF4fix = importcsvDF4fix[importcsvDF4fix['id'].notnull()]
importcsvDF4fix = importcsvDF4fix.apply(pdfield.createRelationFromField, inputfield= 'id', relation ='liesWithin', axis=1)
notfixedsurveyunitsDF = surveyunitsDF[~surveyunitsDF['id'].isin(importcsvDF4fix['id']) ]
importcsvDF4fix.drop(columns=['id'], inplace=True)
importcsvDF4fix.set_index('myid',drop=False,inplace=True)
importcsvDF4.update(importcsvDF4fix)

importcsvDF4fuzzy = pdfield.combineFramesByFuzzyId(targetdf=importcsvDF4fixnot,inputdfex = notfixedsurveyunitsDF,id ='identifier',combinefields = ['id'])
importcsvDF4fuzzy =importcsvDF4fuzzy[importcsvDF4fuzzy['id'].notnull()]
importcsvDF4fuzzy = importcsvDF4fuzzy.apply(pdfield.createRelationFromField, inputfield= 'id', relation ='liesWithin', axis=1)
importcsvDF4fuzzy.drop(columns=['id'], inplace=True)
importcsvDF4fuzzy.set_index('myid',drop=False,inplace=True)
importcsvDF4.update(importcsvDF4fuzzy)

print('Rows after assign SurveyUnits ', len(importcsvDF2))


### Individualize identifier based on REC_NO_ if available###
def newREC_NO(series, idlist):
    series['REC_NO_'] = idlist[0]
    idlist.remove(series['REC_NO_'])
    return series

correctedduplicates = pd.DataFrame()
for number, group in importcsvDF4.groupby(by=['identifier']):
    idlist = list(range(0, len(group)))
    grp = group.groupby(by=['REC_NO_'])
    for number, innergroup in grp:
        if len(innergroup) == 1:
            try:
                idlist.remove(int(number))
            except:
                pass
            correctedduplicates = correctedduplicates.append(innergroup)

    for number, innergroup in grp:
        if len(innergroup) > 1:
            innergroup = innergroup.apply(newREC_NO, idlist=idlist, axis=1)
            correctedduplicates = correctedduplicates.append(innergroup)
print('Correctedduplicates: ', len(correctedduplicates))
correctedduplicates=correctedduplicates.apply(pdfield.formatIdentifier,axis=1)
correctedduplicates.set_index('myid',drop=False,inplace=True)
importcsvDF4.update(correctedduplicates)
print('Check unique identifier:', len(importcsvDF4['identifier'].unique()),'/',len(importcsvDF4['identifier']) )

### Assign isRecordedIn ###
importcsvDF4['recordedInID']= 'baa8937e-39f2-3dc8-b1cb-51566aedf871'
relationsupdate = importcsvDF4[importcsvDF4['relations'].notnull()].apply(pdfield.createRelationFromField, inputfield= 'recordedInID', relation ='isRecordedIn', axis=1)
importcsvDF4.update(relationsupdate)
importcsvDF4.drop(columns=['recordedInID'], inplace=True)
importcsvDF4.drop(columns=['myid'], inplace=True)

### Delete obsolte ###
importcsvDF4.drop(columns=['DIAMETER_I'], inplace=True)
importcsvDF4.drop(columns=['REC_NO_'], inplace=True)





c:\Users\mhaibt\.conda\envs\accessidaifield\lib\site-packages\pandas\core\frame.py:6596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)
c:\Users\mhaibt\Documents\GitHub\pandasField\lib\pdfield.py:397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targetdf[id] = targetdf[id].str.replace(word, replacement, regex=False)


Rows after assign SurveyUnits  6710
Correctedduplicates:  6706
Check unique identifier: 6707 / 6710
Final Rows  6710


KeyError: "['created' 'modified' '_rev' 'project' 'resource.identifier'] not found in axis"

In [21]:
%reload_ext autoreload
%autoreload 2
### Convert to docs ###
print('Final Rows ', len(importcsvDF4))
print(importcsvDF4.columns)
print(docfields)
importcsvDF4['_id'] = importcsvDF4['id']
DOCS = pdfield.DFtoDOCv1(DF=importcsvDF4, docfields=docfields)
DOCS2 = {'docs':[]}
for doc in DOCS['docs']:
    doc = pdfield.enterCreated(doc)
    DOCS2['docs'].append(doc)

Final Rows  6710
Index(['DrawingId', 'FEATURE_NU', 'Firing', 'FormComparisonCompendiadetail',
       'SAMPLE_NUM', 'Unnamed: 21', 'amount', 'clayColorOutside',
       'coatOutsideType', 'condition', 'conditionPercent', 'date',
       'description', 'dimensionDiameter', 'identifier', 'manufacturingMethod',
       'period', 'processor', 'temper', 'temperAmount', 'temperParticles',
       'type', 'vesselpart', 'relations', 'clayColorInside',
       'DecorationComparisonCompendiadetail', '_id', 'id'],
      dtype='object')
['created', 'modified', '_id', '_rev', 'project', 'resource.identifier']
Index(['DrawingId', 'FEATURE_NU', 'Firing', 'FormComparisonCompendiadetail',
       'SAMPLE_NUM', 'Unnamed: 21', 'amount', 'clayColorOutside',
       'coatOutsideType', 'condition', 'conditionPercent', 'date',
       'description', 'dimensionDiameter', 'identifier', 'manufacturingMethod',
       'period', 'processor', 'temper', 'temperAmount', 'temperParticles',
       'type', 'vesselpart', 'relatio

In [ ]:
### Save new data to Field-database ###
#pdfield.bulkSaveChanges(api, docs=DOCS2)